<a href="https://colab.research.google.com/github/rglasnap/python-notebooks/blob/master/Stochastic_Quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
I'm using this notebook to try to understand a talk by Jacob Barandes given on July 9th, 2024 (Brown IDEA Seminar). I'm trying to keep a record of my historical thoughts and false starts in chronological order for my own reference. The only things I plan on moving around are some generic code (like functions).

The Sigma matrix below is one I slightly modified from his example non-markovian stochatic matrix. Slightly modified by reducing it down to a realizeable size. In the talk it was just unspecified by ellipses. So I cut it down to size and added a couple of my own to fill everything in.

Ultimately I'm trying to wrap my mind around what divisible versus indivisible means. And using this as an example to understand that, and the structure/correspondence he's come up with.

#Imports and Functions

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp


In [ ]:
## Sanity checks

def UnitaryChecks(aMatrix):
  print("Unitary checks for:")
  print(aMatrix)
  print("Determinant: ",sp.linalg.det(aMatrix))
  print("A * A^dagger")
  print(np.matmul(aMatrix,aMatrix.conj().T))

  def U(t, Sigma): return sp.linalg.fractional_matrix_power(Sigma, t)
## Lambda is defined by matrix elementwise.
def Lambda(t, Sigma): return np.abs(U(t,Sigma))**2
## Ut_prime is from t' to t
def Ut_prime(t, t_prime, Sigma): return np.matmul(U(t, Sigma),U(t_prime,Sigma).conj().T)
# Lambda_tprime is from t' to t
def Lambda_tprime(t, t_prime, Sigma): return np.abs(Ut_prime(t,t_prime,Sigma))**2

def CompositionCheck(t,t_prime,Sigma):
  ## Composition and divisibility checks
  print("This matrix should not be ~0. Because t' is NOT at a division point")
  print(Lambda(t,Sigma) - np.matmul(Lambda_tprime(t, t_prime, Sigma),Lambda(t_prime,Sigma)))

def DivsibilityCheck(t,t_prime,Sigma):
  print("This matrix should be ~0. Because t' IS at a division point")
  print(Lambda(t,Sigma) - np.matmul(Lambda_tprime(t, t_prime, Sigma),Lambda(t_prime,Sigma)))

# Initial thoughts and tests

In [10]:
Sigma = np.array([[0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 1, 0],
                  [0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0],
                  [1, 0, 0, 0, 0, 0, 0]])

UnitaryChecks(Sigma)

Unitary checks for:
[[0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0]]
Determinant:  1.0
A * A^dagger
[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]]


In [ ]:
CompositionCheck(4.0,1.5,Sigma)

DivsibilityCheck(4.0,2.0,Sigma)




This matrix should not be ~0. Because t' is NOT at a division point
[[-0.04294266  0.65306122 -0.21681392 -0.04294266 -0.21681392 -0.06677403
  -0.06677403]
 [-0.21681392 -0.04294266 -0.04294266  0.65306122 -0.06677403 -0.21681392
  -0.06677403]
 [-0.06677403 -0.06677403 -0.04294266 -0.21681392 -0.21681392  0.65306122
  -0.04294266]
 [-0.06677403 -0.21681392  0.65306122 -0.04294266 -0.06677403 -0.04294266
  -0.21681392]
 [ 0.65306122 -0.04294266 -0.06677403 -0.21681392 -0.04294266 -0.06677403
  -0.21681392]
 [-0.21681392 -0.06677403 -0.21681392 -0.06677403 -0.04294266 -0.04294266
   0.65306122]
 [-0.04294266 -0.21681392 -0.06677403 -0.06677403  0.65306122 -0.21681392
  -0.04294266]]
This matrix should be ~0. Because t' IS at a division point
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


(Note from future self): Something about this was bugging me when I originally went through it. So I decided to take a look at some explicitly markovian processes to see how they behaved following this same recipe.

Looking into Markov Chains explicitly and using an example from Brilliant.org.

Conceptually the idea of memorylessness makes sense, but it's always good to practice with something simple to make sure you actually get the concept (which is how this whole thing started anyway.

Explicitly in text:
* A to A: 0.3
* A to B: 0.7
* B to A: 0.8
* B to B: 0.2

Cite:
*Markov Chains*. Brilliant.org. Retrieved 11:48, January 23, 2025, from https://brilliant.org/wiki/markov-chains/

In [ ]:
Markov = np.array( [[0.3,0.7],[0.8,0.2] ])

UnitaryChecks(Markov)

Unitary checks for:
[[0.3 0.7]
 [0.8 0.2]]
Determinant:  -0.5
A * A^dagger
[[0.58 0.38]
 [0.38 0.68]]


In [ ]:
np.matmul(Markov,Markov)

array([[0.65, 0.35],
       [0.4 , 0.6 ]])

Ok, so not Unitary and the transition matrix for 2 steps checks out with what's on Brilliant (I.e 2 steps from A to B is 0.35).

Out of curiosity lets try a slightly different matrix.

In [ ]:
Markov2 = np.array( [[0.3,0.7],[0.7,0.3] ])

UnitaryChecks(Markov2)

Unitary checks for:
[[0.3 0.7]
 [0.7 0.3]]
Determinant:  -0.39999999999999997
A * A^dagger
[[0.58 0.42]
 [0.42 0.58]]




Lets try a different one from brilliant. A to C to B, basic loop. Why this one? Because it's smaller, but fundamentally similar to the original Sigma matrix I started with at the top. And I'm trying to understand what in particular makes THAT one non-markovian.

In [ ]:
Markov_small = np.array( [[0, 0, 1],[1, 0, 0],[0, 1, 0]])

UnitaryChecks(Markov_small)

CompositionCheck(4.0,1.5,Markov_small)

DivsibilityCheck(4.0,2.0,Markov_small)

Unitary checks for:
[[0 0 1]
 [1 0 0]
 [0 1 0]]
Determinant:  1.0
A * A^dagger
[[1 0 0]
 [0 1 0]
 [0 0 1]]
This matrix should not be ~0. Because t' is NOT at a division point
[[-0.2962963  -0.2962963   0.59259259]
 [ 0.59259259 -0.2962963  -0.2962963 ]
 [-0.2962963   0.59259259 -0.2962963 ]]
This matrix should be ~0. Because t' IS at a division point
[[0 0 0]
 [0 0 0]
 [0 0 0]]


Ok, so this one is clearly markovian, but works through the process just fine.

Also, I think my original matrix is markovian as well as it functions similarly the same.  

I found a nice example of a non-markovian process (https://math.stackexchange.com/questions/89394/example-of-a-stochastic-process-which-does-not-have-the-markov-property) with the green and red balls. That one very clearly has memeory, but I've no idea how to turn it into a stochastic matrix.


June 2, 2025 Note: Ok so I think I figured out what I was missing initially, and why it was bugging me. My first sigma matrix is doesn't change in time. Configuration one does transition to configuration seven. But it ALWAYS does that and I was treating successive applications of the matrix as time steps.

But then, how do you take that and make a permutation matrix out of it.